<a href="https://colab.research.google.com/github/happydracula/eBPF-Firewall/blob/main/ML/malicious_ip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import math

In [2]:
def encodeIP(ip_addr,N):
    X=ip_addr.split('.')
    encoding=[0 for i in range(256*N)]
    for i in range(1,N+1,1):
        k=256*(i-1)+int(X[i-1])
        #print(k)
        encoding[k]=1
    return encoding
np.shape(encodeIP('198.51.100.88',3))

(768,)

In [3]:
def get_from_file(filepath):
    f = open(filepath, "r")
    ip_list=f.read().split('\n')[:-1]
    ip_encodings=[]
    idx=int(filepath[-5])
    print(str(idx)+':'+str(len(ip_list)))
    if(idx>=1):
      idx=1

    for ipaddr in ip_list:
        ip_encoding=encodeIP(ipaddr,3)
        ip_encoding.append(idx)
        ip_encodings.append(ip_encoding)

    return ip_encodings
data=get_from_file('./levels/8.txt')

8:14


In [4]:
np.shape(data)


(14, 769)

In [5]:
def get_from_folder(path):
    dataset=[]
    for i in range(0,9):
        data=get_from_file(os.path.join(path,str(i)+'.txt'))
        for encoding in data:
            dataset.append(encoding)
    return dataset
dataset=get_from_folder('./levels')

0:300000
1:239625
2:29734
3:12804
4:5964
5:2295
6:577
7:90
8:14


In [ ]:
np.shape(dataset)

In [6]:
random.shuffle(dataset)
dataset=dataset[:300000]

In [ ]:
l=[[1,2,3],[4,5,6]]
l=np.array(l)
l[:,:-1]

array([[1, 2],
       [4, 5]])

In [7]:
dataset=np.array(dataset)
ratio=int(0.8*np.shape(dataset)[0])
train=dataset[:ratio,:]
test=dataset[ratio:,:]


In [11]:
del dataset

NameError: ignored

In [12]:
train_X=train[:,:-1]
train_Y=train[:,-1]
test_X=test[:,:-1]
test_Y=test[:,-1]

In [16]:
np.shape(train_Y)

(240000,)

In [13]:
del train
del test

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [30]:
model.fit(train_X, train_Y, epochs=5)

Epoch 1/5
7500/7500 [==============================] - 13s 2ms/step - loss: 0.5954 - accuracy: 0.6943
Epoch 2/5
7500/7500 [==============================] - 12s 2ms/step - loss: 0.5378 - accuracy: 0.7134
Epoch 3/5
7500/7500 [==============================] - 12s 2ms/step - loss: 0.5286 - accuracy: 0.7150
Epoch 4/5
7500/7500 [==============================] - 12s 2ms/step - loss: 0.5265 - accuracy: 0.7158
Epoch 5/5
7500/7500 [==============================] - 12s 2ms/step - loss: 0.5260 - accuracy: 0.7158


In [31]:
model.evaluate(test_X,test_Y)

1875/1875 [==============================] - 3s 1ms/step - loss: 0.5275 - accuracy: 0.7140


[0.5274689197540283, 0.7139666676521301]

In [55]:
encoding=encodeIP('61.177.172.179',3)
model.predict(np.array([encoding]))

1/1 [==============================] - 0s 25ms/step


array([[0.61007154]], dtype=float32)

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1)                 769       
                                                                 
Total params: 769 (3.00 KB)
Trainable params: 769 (3.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
for layer in model.layers: print(layer.get_config(), layer.get_weights())

In [41]:
weights = model.layers[0].get_weights()[0]
biases  = model.layers[0].get_weights()[1]

In [45]:
weights=weights*1000000000
biases=biases*1000000000



In [47]:
weights=weights.astype('int64')
biases=biases.astype('int64')

In [56]:
res=0
for i in range(0,768):
  res=res+weights[i]*encoding[i]
res+=biases[0]

In [57]:
x=res/1000000000
pred=1/(1+math.exp(-x))
print(pred)

0.6100715220208411
